##Solutions to Practice Example Chapter 7##

In [1]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
import numpy as np
import pysal

In [3]:
db = pysal.open('data/south.dbf','r')

In [4]:
len(db)

1412

In [5]:
db.header

['FIPSNO',
 'NAME',
 'STATE_NAME',
 'STATE_FIPS',
 'CNTY_FIPS',
 'FIPS',
 'STFIPS',
 'COFIPS',
 'SOUTH',
 'HR60',
 'HR70',
 'HR80',
 'HR90',
 'HC60',
 'HC70',
 'HC80',
 'HC90',
 'PO60',
 'PO70',
 'PO80',
 'PO90',
 'RD60',
 'RD70',
 'RD80',
 'RD90',
 'PS60',
 'PS70',
 'PS80',
 'PS90',
 'UE60',
 'UE70',
 'UE80',
 'UE90',
 'DV60',
 'DV70',
 'DV80',
 'DV90',
 'MA60',
 'MA70',
 'MA80',
 'MA90',
 'POL60',
 'POL70',
 'POL80',
 'POL90',
 'DNL60',
 'DNL70',
 'DNL80',
 'DNL90',
 'MFIL59',
 'MFIL69',
 'MFIL79',
 'MFIL89',
 'FP59',
 'FP69',
 'FP79',
 'FP89',
 'BLK60',
 'BLK70',
 'BLK80',
 'BLK90',
 'GI59',
 'GI69',
 'GI79',
 'GI89',
 'FH60',
 'FH70',
 'FH80',
 'FH90']

In [6]:
y_name = 'HR90'
y = np.array([db.by_col(y_name)]).T

In [7]:
x_names = ['RD90','MA90','PS90','UE90']
x = np.array([db.by_col(var) for var in x_names]).T

In [8]:
xe_names = ['RD90','MA90','PS90']
xe = np.array([db.by_col(var) for var in xe_names]).T

In [9]:
yend_names = ['UE90']
yend = np.array([db.by_col(var) for var in yend_names]).T

In [10]:
q_names = ['FH90','FP89','GI89']
q = np.array([db.by_col(var) for var in q_names]).T

In [11]:
w = pysal.knnW_from_shapefile('data/south.shp',
                                k=6,radius=pysal.cg.RADIUS_EARTH_MILES,idVariable='FIPSNO')
w.transform = 'r'

In [12]:
ww = pysal.queen_from_shapefile('data/south.shp',idVariable="FIPSNO")
ww.transform = 'r'

In [13]:
w.n, ww.n

(1412, 1412)

In [14]:
kw = pysal.adaptive_kernelW_from_shapefile('data/south.shp',
                                             k=12,function='quadratic',radius=pysal.cg.RADIUS_EARTH_MILES,
                                             diagonal=True,idVariable='FIPSNO')

In [15]:
kw.n

1412

spatial two stage least squares: exogenous variables only, using k=6 nearest neighbors

In [16]:
reg1 = pysal.spreg.GM_Lag(y,x,w=w,spat_diag=True,
                          name_y=y_name,name_x=x_names,
                          name_w='south_k6',name_ds='south')

In [17]:
print reg1.summary

REGRESSION
----------
SUMMARY OF OUTPUT: SPATIAL TWO STAGE LEAST SQUARES
--------------------------------------------------
Data set            :       south
Weights matrix      :    south_k6
Dependent Variable  :        HR90                Number of Observations:        1412
Mean dependent var  :      9.5493                Number of Variables   :           6
S.D. dependent var  :      7.0389                Degrees of Freedom    :        1406
Pseudo R-squared    :      0.3170
Spatial Pseudo R-squared:  0.3017

------------------------------------------------------------------------------------
            Variable     Coefficient       Std.Error     z-Statistic     Probability
------------------------------------------------------------------------------------
            CONSTANT       8.7095236       2.0183412       4.3151890       0.0000159
                MA90       0.0144354       0.0480777       0.3002507       0.7639859
                PS90       2.0300519       0.2069009       

spatial two stage least squares using k=6 nearest neighbors, with HAC standard errors

In [18]:
reg1a = pysal.spreg.GM_Lag(y,x,w=w,robust='hac',gwk=kw,spat_diag=True,
                          name_y=y_name,name_x=x_names,
                          name_w='south_k6',name_gwk="south_quad_12",name_ds='south')

In [19]:
print reg1a.summary

REGRESSION
----------
SUMMARY OF OUTPUT: SPATIAL TWO STAGE LEAST SQUARES
--------------------------------------------------
Data set            :       south
Weights matrix      :    south_k6
Dependent Variable  :        HR90                Number of Observations:        1412
Mean dependent var  :      9.5493                Number of Variables   :           6
S.D. dependent var  :      7.0389                Degrees of Freedom    :        1406
Pseudo R-squared    :      0.3170
Spatial Pseudo R-squared:  0.3017

HAC Standard Errors; Kernel Weights: south_quad_12
------------------------------------------------------------------------------------
            Variable     Coefficient       Std.Error     z-Statistic     Probability
------------------------------------------------------------------------------------
            CONSTANT       8.7095236       2.2063492       3.9474820       0.0000790
                MA90       0.0144354       0.0493446       0.2925419       0.7698723
        

spatial two stage least squares using queen contiguity

In [20]:
reg1b = pysal.spreg.GM_Lag(y,x,w=ww,spat_diag=True,
                          name_y=y_name,name_x=x_names,
                          name_w='south_queen',name_ds='south')

In [21]:
print reg1b.summary

REGRESSION
----------
SUMMARY OF OUTPUT: SPATIAL TWO STAGE LEAST SQUARES
--------------------------------------------------
Data set            :       south
Weights matrix      : south_queen
Dependent Variable  :        HR90                Number of Observations:        1412
Mean dependent var  :      9.5493                Number of Variables   :           6
S.D. dependent var  :      7.0389                Degrees of Freedom    :        1406
Pseudo R-squared    :      0.3222
Spatial Pseudo R-squared:  0.3020

------------------------------------------------------------------------------------
            Variable     Coefficient       Std.Error     z-Statistic     Probability
------------------------------------------------------------------------------------
            CONSTANT       8.1830219       2.0412020       4.0089231       0.0000610
                MA90       0.0227990       0.0483737       0.4713098       0.6374195
                PS90       2.0297011       0.2037269       

spatial two stage least squares with exogenous and endogenous variables, k=6 nearest neighbors

In [22]:
reg2 = pysal.spreg.GM_Lag(y,xe,yend,q,w=w,spat_diag=True,
                          name_y=y_name,name_x=xe_names,name_yend=yend_names,
                          name_q=q_names,name_w='south_k6',name_ds='south')

In [23]:
print reg2.summary

REGRESSION
----------
SUMMARY OF OUTPUT: SPATIAL TWO STAGE LEAST SQUARES
--------------------------------------------------
Data set            :       south
Weights matrix      :    south_k6
Dependent Variable  :        HR90                Number of Observations:        1412
Mean dependent var  :      9.5493                Number of Variables   :           6
S.D. dependent var  :      7.0389                Degrees of Freedom    :        1406
Pseudo R-squared    :      0.2800
Spatial Pseudo R-squared:  0.2774

------------------------------------------------------------------------------------
            Variable     Coefficient       Std.Error     z-Statistic     Probability
------------------------------------------------------------------------------------
            CONSTANT      14.4390815       2.2781115       6.3381803       0.0000000
                MA90      -0.0170951       0.0499350      -0.3423476       0.7320893
                PS90       2.2209207       0.2158787      1